# Scope of this tutorial

This tutorial is going to walk through adding a new component to represent *Aquaculture*.  Aquaculture represents about 7% of total water demand (USGS 2010).  The USGS splits out aquaculture in its 5-year reports, but since aquaculture water use is determined by a combination of production and evaporation, we can model future changes.

![Aquaculture](http://2.bp.blogspot.com/--s363YMOKpg/UE41k-l0RhI/AAAAAAAAAJw/TQw23AAjwrY/s1600/Tsar+Nicolai+Tank+Farm.jpg)

The hope is that this will provide a basis for understanding how to add new demands in general, and other kinds components as well.

# Starting a git branch

The first step to making a new component is always starting a new *branch* of the code.  Branches allow parallel lines of work to be managed, and then incorporated together when they are ready.

In [18]:
run(`git checkout -b aquacomp`)

M	configs/standard-1year.yml
M	configs/standard.yml
A	prepare/netcdf2csv.jl
M	src/lib/datastore-netcdf.jl
M	src/lib/datastore-nonetcdf.jl
M	src/optimize-surface.jl
M	src/optimize.jl
D	src/tools/netcdf2csv.jl


Switched to a new branch 'aquacomp'


In [17]:
`git status`

`git status`

# Components of a component

Most components have a similar structure, as follows:
1. A comment block describing the component.
2. Loading required modules.
3. The component definition.
4. The simulation timestep function.
5. The initialization function.
6. The optimization gradients.

After that is done, we need to do some things outside of the individual component.
* Create a documentation notebook, showing validation and example results.
* Create an automated "unit test", testing proper functionality.
* Connect the component to other components in the simulation and optimization.

We will consider each of these in order.

# Commenting the script

Make sure to use both normal comments, which start with `#`, and [docstrings](http://docs.julialang.org/en/stable/manual/documentation/).  Here is the comment at the top of the file.

In [1]:
# The Freshwater Aquaculture component.
#
# It uses aquaculture production data as a baseline, which is then allowed to change with temperature and production.

# Required modules

The only module that we always need is `Mimi`.  We also need to read data (`DataFrames` and `lib/datastore`).

In [2]:
using DataFrames
using Mimi
include("lib/datastore.jl")

LoadError: LoadError: could not open file /Users/jrising/research/water/model/operational-problem/docs/lib/datastore.jl
while loading In[2], in expression starting on line 3

# Component definition

The component definition starts with an `@defcomp` line and ends with `end`.  In between are three parts:
1. Indexes, which define the dimensions that are used by parameters and components.
2. Parameters, which are the inputs to the component.
3. Variables, which are the outputs.

In [3]:
@defcomp Aquaculture begin
    regions = Index()

    # The basic pattern of aquaculture (could vary in time)
    demand_baseline = Parameter(index=[regions, time], unit="1000 m^3")

    # The effect of production on demand
    production_baseline = Parameter(index=[time], unit="MT") # 299,396 in 2010
    production = Parameter(index=[time], unit="MT")

    # Demand combining the two effects
    demand = Variable(index=[regions, time], unit="1000 m^3")
end

Aquaculture

# Simulation timestep function

The `run_timestep` specifies how variables can be computed from parameters in each timestep.

In [4]:
function run_timestep(c::Aquaculture, tt::Int)
    v, p, d = getvpd(c)

    # Scale with production
    v.demand[:, tt] = (p.production[tt] / p.production_baseline[tt]) * p.demand_baseline[:, tt]
end

run_timestep (generic function with 3 methods)

# Initialization function

The initialization function is not strictly a part of the Mimi framework, but I find it useful.  It creates the component, by adding it to the model and filling in all the parameters.

In [5]:
function initaquaculture(m::Model)
    aquaculture = addcomponent(m, Aquaculture);

    scaling = config["timestep"] / 12.

    # Baseline from USGS
    aquaculture[:demand_baseline] = repeat(convert(Vector{Float64}, readtable(datapath("aquaculture/usgsextract.csv"))[:AQ_WFrTo]) * scaling, outer=[1, m.indices_counts[:time]])

    # Production data from Fisheries of the United States
    production = readtable(datapath("aquaculture/production.csv"))

    aquaculture[:production_baseline] = repeat(production[production[:year] .== 2010, :production] * scaling, outer=[numsteps])
    aquaculture[:production] = repeat(production[production[:year] .>= 2010, :production] * scaling, inner=[round(Int64, 1. / scaling)])

    aquaculture
end

initaquaculture (generic function with 1 method)

Up to this point, these are all values that go into the component file itself.  Now let's switch to the unit test file, as we try it out.  We make a new file `test/test_Aquaculture.jl`.

Here's what the `test_Aquaculture.jl` file looks like:

In [7]:
# Load the component
#include("../src/SimpleAquaculture.jl")
include("../src/lib/datastore.jl")
include("../src/lib/readconfig.jl")

# Set up the model
counties = convert(Vector{Float64}, readtable(datapath("aquaculture/usgsextract.csv"))[:,:FIPS])

config = readconfig("../configs/single.yml")
numsteps = 1
numcounties = length(counties)

m = Model()
setindex(m, :time, [1]) # Single period
setindex(m, :regions, counties)

# Add the component
initaquaculture(m)

# Run the model
run(m)

# Check that it matches 2010
demand_baseline = repeat(convert(Vector{Float64}, readtable(datapath("aquaculture/usgsextract.csv"))[:AQ_WFrTo]), outer=[1, m.indices_counts[:time]])
println(m[:Aquaculture, :demand] - demand_baseline)

1.0
[0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 

Now we need to embed this into the rest of the model.  If you look in `model.jl`, there are a sequence of `include` lines for each component, and then `init` lines.  The order of the `init` lines matters, because this is that order that components will be computed, and each component needs its dependent components to already have been computed.

However, the only imput to this component is the level of production, which is set by the optimization or simulation information.

In [9]:
include("../src/model2.jl")

Loading from saved region network...


Loading from saved water network...
Loading from saved region network...
Creating model...


Mimi.VariableReference(Mimi.Model(Dict(:regions=>3109,:crops=>9,:time=>1,:edges=>18561,:canals=>21598,:aquifers=>3108,:gauges=>22559,:reservoirs=>2671),Dict(:regions=>Any["01001","01003","01005","01007","01009","01011","01013","01015","01017","01019"  …  "56027","56029","56031","56033","56035","56037","56039","56041","56043","56045"],:crops=>Any["alfalfa","otherhay","Barley","Barley.Winter","Maize","Sorghum","Soybeans","Wheat","Wheat.Winter"],:time=>Any[24106],:edges=>Any[1,2,3,4,5,6,7,8,9,10  …  18552,18553,18554,18555,18556,18557,18558,18559,18560,18561],:canals=>Any[1,2,3,4,5,6,7,8,9,10  …  21589,21590,21591,21592,21593,21594,21595,21596,21597,21598],:aquifers=>Any[1,2,3,4,5,6,7,8,9,10  …  3099,3100,3101,3102,3103,3104,3105,3106,3107,3108],:gauges=>Any["usgs.02456000","junction.13805-dn","junction.11142-up","usgs.03488000","usgs.12113347","junction.13981-up","junction.12762-up","usgs.0214627970","usgs.01396800","usgs.05532000"  …  "reservoir.1656","usgs.02374950","usgs.02110701","re

Now let's consider optimization.  Like agriculture, aquaculture production can go into the producer-viewed objective function.